In [1]:
import pandas as pd
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import boto3
from boto3.s3.transfer import TransferConfig
import os

# Unzip Files of Labels

In [4]:
# import zipfile

# with zipfile.ZipFile('chapter_labels.zip', 'r') as zip_ref:
#     zip_ref.extractall('chapter_labels')

# Helper Functions

In [5]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [6]:
def LoadFromS3(i):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Load for File",i)
    embeddings = {}
    session = boto3.session.Session(region_name='us-east-1')
    s3client = session.client('s3')
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
    with open(i, 'wb') as handle:
        pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("File Loaded in:", time.time()-now)
    
    return embeddings

In [7]:
def SaveToS3(file, name):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Write for File", name)
    file = file.to_json()
    file.save(f'{name}.h5')
    session = boto3.session.Session(region_name='us-east-1')
    s3 = boto3.resource('s3')
    object = s3.Object('w210-mimic', name)
    object.put(Body=file)

    print("File Saved in:", time.time()-now)
    
    return

In [7]:
# # Saving the Files Locally:

# model_json = model.to_json()

# with open("adverse_num.json", "w") as json_file:
#     json_file.write(model_json)

# # serialize weights to HDF5
# model.save_weights("adverse_num.h5")

# Initialize the Models

In [11]:
model_list = []

for file in os.listdir('./chapter_labels/'):
    LABEL_DF = pd.read_csv('./chapter_labels/'+file, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    
    NODE_COUNT = len(LABEL_DF.columns)
    
    deep_inputs = Input(shape=(3840,768))
    LSTM_Layer_1 = LSTM(128)(deep_inputs)
    dense_layer_1 = Dense(NODE_COUNT, activation='sigmoid')(LSTM_Layer_1)
    opt = keras.optimizers.Adam(lr=0.0001)
    model = Model(inputs=deep_inputs, outputs=dense_layer_1)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])
    model_list.append(model)
    
    model_json = model.to_json()
    with open(f"models/model_{file}.json", "w") as json_file:
        json_file.write(model_json)

In [12]:
for model in model_list:
    print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 3840, 768)         0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 128)               459264    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1290      
Total params: 460,554
Trainable params: 460,554
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 3840, 768)         0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)               459264    
_________________________________________________________________

# Create List of Models with outputs that tie to Chapter Labels

In [13]:
# from keras.models import model_from_json

# json_file = open('./adverse_num.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# loaded_model.load_weights("./adverse_3_embeddings1.hdr")

# opt = keras.optimizers.Adam(lr=0.0001)
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

# print("Loaded model from disk")

In [14]:
EPOCHS = 5

In [ ]:
for epoch in range(EPOCHS):
    for i in ['embeddings6', 'embeddings5', 'embeddings4', 'embeddings3', 'embeddings2', 'embeddings1']:
        rawData = LoadFromS3(i)
        
        rawData = np.array(list(rawData.items()))
        HADM_ID_LIST_RAW = rawData[:,0]
        rawData = rawData[:,1]

        
        for model_index, file in enumerate(os.listdir('./chapter_labels/')):
            LABEL_DF = pd.read_csv('./chapter_labels/'+file, sep=',', header = 0)\
                                .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
        
            # Subset the Raw Data to only include records present in the LABEL_DF
            matchingIndex = np.in1d(HADM_ID_LIST_RAW, LABEL_DF.index)
            
            data = rawData[matchingIndex]
            HADM_ID_LIST = HADM_ID_LIST_RAW[matchingIndex]
            del(matchingIndex)
            
            data = [np.float16(np.concatenate(i[:30])) for i in data]
            embeddings_padded = []
        
            print("Padding Embeddings:")
            for j in data:
                pad_len = 3840 - len(j)
                if pad_len:
                    embeddings_padded.append(np.append(j, np.zeros(pad_len*768, dtype=np.float16).reshape(pad_len, 768), axis=0))
                else:
                    embeddings_padded.append(j)

            del(data)
            embeddings_padded = np.float16(embeddings_padded)
            embeddings_padded = np.array(embeddings_padded, dtype=np.float16)

            EMBEDDINGS_CHAPTER_PIVOT_DF = pd.DataFrame(data=list(zip(HADM_ID_LIST, embeddings_padded)), columns=['HADM_ID', 'embeddings']).set_index('HADM_ID')\
                        .join(LABEL_DF, how='left')
            EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.fillna(0)
            EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.iloc[:,1:]
            EMBEDDINGS_CHAPTER_PIVOT_NP = np.array(EMBEDDINGS_CHAPTER_PIVOT_DF)
            del(EMBEDDINGS_CHAPTER_PIVOT_DF)

            # Rebalance the Dataset using class weights
    #         class_weights = {}
    #         class_weight_vals = 1/(EMBEDDINGS_CHAPTER_PIVOT_NP.sum(axis=0)/EMBEDDINGS_CHAPTER_PIVOT_NP.shape[0])

    #         for index, weight in enumerate(class_weight_vals):
    #             class_weights[index] = int(weight)

            print(f"\nTraining Model: {file} Epoch: {epoch} Embeddings: {i}")
            current_model = model_list[model_index]
            
            history = current_model.fit(embeddings_padded, EMBEDDINGS_CHAPTER_PIVOT_NP, epochs=1, batch_size=256, verbose=1, validation_split=0.1) # ,class_weight=class_weights)
            current_model.save(f"models/model_{file}_{epoch}_{i}.hdr")


Starting Load for File embeddings6
File Loaded in: 678.4929823875427
Padding Embeddings:

Training Model: 630_679__740_759__760_779.csv Epoch: 0 Embeddings: embeddings6



Train on 79 samples, validate on 9 samples
Epoch 1/1





79/79 [==============================] - 49s 623ms/step - loss: 0.6905 - get_f1: 0.0099 - val_loss: 0.6677 - val_get_f1: 0.0909
Padding Embeddings:

Training Model: 710-739.csv Epoch: 0 Embeddings: embeddings6
Train on 607 samples, validate on 68 samples
Epoch 1/1
607/607 [==============================] - 45s 74ms/step - loss: 0.6550 - get_f1: 0.0999 - val_loss: 0.6127 - val_get_f1: 0.1176
Padding Embeddings:

Training Model: 780-799.csv Epoch: 0 Embeddings: embeddings6
Train on 1066 samples, validate on 119 samples
Epoch 1/1
1066/1066 [==============================] - 68s 63ms/step - loss: 0.7005 - get_f1: 0.0481 - val_loss: 0.6483 - val_get_f1: 0.0471
Padding Embeddings:

Training Model: 290_319.csv Epoch: 0 Embeddings: embeddings6
Train on 839 samples, va

# Cells Below are for Debugging & Legacy

In [17]:
2+2

4

In [14]:
who

Activation	 Concatenate	 Dense	 Dropout	 EMBEDDINGS_CHAPTER_PIVOT_NP	 EPOCHS	 Embedding	 Flatten	 GlobalMaxPooling1D	 
HADM_ID_LIST	 HADM_ID_LIST_RAW	 Input	 K	 LABEL_DF	 LSTM	 LSTM_Layer_1	 LoadFromS3	 Model	 
NODE_COUNT	 Reshape	 SaveToS3	 Sequential	 Tokenizer	 TransferConfig	 boto3	 cPickle	 current_model	 
deep_inputs	 dense_layer_1	 embeddings_padded	 epoch	 file	 get_f1	 history	 i	 io	 
j	 keras	 matchingIndex	 model	 model_index	 model_list	 np	 one_hot	 opt	 
os	 pad_len	 pad_sequences	 pd	 pickle	 plt	 rawData	 tf	 time	 
train_test_split	 


In [19]:
rawData.shape

(10000,)